# Multimodal (2D/3D) change detection for natural disaster response </h1>

**This tutorial will highlight the capabilities of multimodal satellite imagery in addressing some of nowadays most impacting societal challenges using open source tools from CNES (French space agency).**

In anticipation of the future CO3D mission, the French space agency (CNES) is currently developing open source tools for large-scale 3D data processing. With the arrival of 3D data on a more frequent basis, new use-cases for such data sources are emerging and enabling its applicability to pressing challenges such as improved crisis management in the aftermath of natural disasters. Change detection methodologies and their application to natural disaster response constitute a long-standing field within the remote sensing community, with initiatives like The International Charter Space and Major Disasters, open data programs of commercial satellite providers or annotated datasets like xview2 enabling a growing number of researchers and practitioners to continuously improve existing solutions. However, until now, the availability of post-event imagery, and thus related algorithms, have mainly been limited to monoscopic acquisitions, whereas with this type of constellations, a growing number of 3D data should also cover areas impacted by climate-related hazards.

In this context, CNES is currently working on the hybridization of 2D and 3D data for multimodal change detection. The proposed tutorial will present and teach some of this work being carried out by CNES through a pipeline that can be used for rapid mapping and longer-term risk and recovery management in order to help improving and enriching common existing approaches.

During this tutorial, you will discover how to:
- generate Digital Surface Models from sets of stereo satellite images using the CARS 3D reconstruction library : https://github.com/CNES/cars
- extract the Digital Terrain Models from Digital Surface Models through the Bulldozer library and derive a final Digital Height Model based on the obtained results : https://github.com/cnes/bulldozer
- extract semantic information from the 2D imagery by using classification models and spectral indices
- combine 2D and 3D change indicators in order to quantify and localize the potentially changed areas on a map
- visualize, filter and extract information on these changes using the uncertainties provided by the tools

## Installation

### Download software

In [ ]:
!rm -rf outresults config.json config.yaml

In [ ]:
!python --version

In [ ]:
import os
if os.path.exists("requirements.txt") is False:
    !wget https://raw.githubusercontent.com/cars-cnes/change-detection-for-natural-disaster-response/main/requirements.txt
    !pip install -r requirements.txt

In [ ]:
!cars -h

In [ ]:
!bulldozer -h

### Useful functions

#### Display function

In [ ]:
def normalize(array, quantile=2):
    """
    normalizes bands into 0-255 scale
    :param array: numpy array to normalize
    :param quantile: quantile for ignoring outliers
    """
    array = np.nan_to_num(array)
    array_min, array_max = np.percentile(array, quantile), np.percentile(array, 100-quantile)
    normalized = 255*((array - array_min)/(array_max - array_min))
    normalized[normalized>255] = 255
    normalized[normalized<0] = 0
    return normalized.astype(np.uint8)

#### Morphologic filter

In [ ]:
from skimage import morphology
import cv2
import numpy as np

def opening_by_reconstruction(pict, kernel):
    reconstruction = cv2.morphologyEx(pict, cv2.MORPH_OPEN, kernel)
    while True:
        dilated = np.minimum(pict, cv2.dilate(reconstruction, kernel))
        if (dilated == reconstruction).all():
            return reconstruction
        reconstruction = dilated

kernel_size=15
kernel_type='square'

kernel = np.ones((kernel_size, kernel_size), np.uint8)

### Download data

In [ ]:
from matplotlib import pyplot
import rasterio
from rasterio.plot import show

!mkdir -p data
for pair in ["pre", "post"]:
    for mod in ["img", "color"]:
        for idx in range(2):
            for ext in [".geom", ".tif"]:
                filename = "data/"+pair+"_event_"+mod+str(idx+1)+ext
                if os.path.exists(filename) is False:
                    filename = "https://raw.githubusercontent.com/cars-cnes/change-detection-for-natural-disaster-response/main/"+filename
                    !wget {filename} -P data/

In [ ]:
fig, axs = pyplot.subplots(2, 2, figsize=(10,10))

for idx1, pair in enumerate(["pre", "post"]):
    for idx2 in range(2):
        with rasterio.open("data/"+pair+"_event_img"+str(idx2+1)+".tif") as src:
            show((src), ax=axs[idx1, idx2], cmap='gray', title=pair+"-event "+str(idx2+1))

## Generate Digital Surface Models from sets of stereo satellite images
using the CARS 3D reconstruction library : https://github.com/CNES/cars


### Create your configuration file

CARS minimal configuration

In [ ]:
sensors = dict()
sensors["pre1"] = {"image": "data/pre_event_img1.tif",
                   "geomodel": "data/pre_event_img1.geom"}
sensors["pre2"] = {"image": "data/pre_event_img2.tif",
                   "geomodel": "data/pre_event_img2.geom"}

pairing = list()
pairing.append(["pre1", "pre2"])

inputs = {"sensors": sensors, "pairing": pairing}

config = {}
config["inputs"] = inputs
config["output"] = {"out_dir": "outresults/dsmpre"}

Adding colors for True Ortho

In [ ]:
config["inputs"]["sensors"]["pre1"]["color"] = "data/pre_event_color1.tif"
config["inputs"]["sensors"]["pre2"]["color"] = "data/pre_event_color2.tif"

Adding ROI

In [ ]:
config["inputs"]["roi"] = {"type": "FeatureCollection",
                           "features": [{"type": "Feature", "properties": {},
                                         "geometry": {"coordinates":
                                                      [[[36.905, 37.588],
                                                        [36.905, 37.581],
                                                        [36.913, 37.581],
                                                        [36.913, 37.588],
                                                        [36.905, 37.588]]],
                                                      "type": "Polygon"}}]}

Saving the epipolar images

In [ ]:
config["applications"] = {}
config["applications"]["resampling"] = {"save_epipolar_image": True}

Saving correlator confidence

In [ ]:
config["applications"]["point_cloud_rasterization"] = {"save_confidence": True}

### Launch CARS - Python API

In [ ]:
import os
from cars.pipelines.pipeline import Pipeline

pipeline = Pipeline("sensors_to_dense_dsm", config, os.getcwd())
pipeline.run()

### Let's see the results

#### Open & format output data

In [ ]:
import rasterio
import numpy as np

with rasterio.open('outresults/dsmpre/dsm.tif') as dsm_reader:
  altitudes = dsm_reader.read(1)
  transform = dsm_reader.transform
  width, height = dsm_reader.width, dsm_reader.height
  cols, rows = np.meshgrid(np.arange(width), np.arange(height))

  # get coordinates to plot points cloud
  x_coords, y_coords = rasterio.transform.xy(transform, rows, cols, offset='center')
  x_coords = np.ravel(x_coords).T
  y_coords = np.ravel(y_coords).T
  z_coords = altitudes.reshape(-1).T

  nodata = dsm_reader.nodata

with rasterio.open('outresults/dsmpre/clr.tif') as clr_reader:
  pre_event_colors = clr_reader.read().astype(float)
  pre_event_colors = pre_event_colors[:3,:,:].transpose(1,2,0)

# stack coords as points cloud
cloud = np.stack((x_coords, y_coords, z_coords), axis=1)
valid = cloud[:, 2] != nodata
cloud = cloud[valid]

# remove nodata altitudes
altitudes[altitudes==nodata] = np.nan

#### Check epipolar images

In [ ]:
epi_img_left_path = "outresults/dsmpre/pre1_pre2/epi_img_left.tif"
epi_img_right_path = "outresults/dsmpre/pre1_pre2/epi_img_right.tif"

epi_img_left = rasterio.open(epi_img_left_path).read(1)
epi_img_right = rasterio.open(epi_img_right_path).read(1)

fig1, (ax1, ax2) = pyplot.subplots(1,2, sharex=True, sharey=True,  figsize=(15,9))
ax1.imshow(normalize(epi_img_left), cmap='gray')
ax2.imshow(normalize(epi_img_right), cmap='gray')
ax1.set_title('left-epipolar image', fontsize=15)
ax2.set_title('right-epipolar image', fontsize=15)
ax1.plot([0, epi_img_right.shape[0]], [240,240], color="red", linewidth=2)
ax2.plot([0, epi_img_right.shape[0]], [240,240], color="red", linewidth=2)
pyplot.show()

#### Results visualization (2D)

In [ ]:
fig, (ax1, ax2) = pyplot.subplots(nrows=1, ncols=2, figsize=(15, 9),
                              constrained_layout=True)

fig.suptitle("Altitudes and Colors as images", fontsize=15)
im1 = ax1.imshow(altitudes, cmap='jet', aspect="auto")
ax1.axis('off')
ax1.set_title('altitudes', fontsize=15)
ax2.imshow(normalize(pre_event_colors), aspect="auto")
ax2.axis('off')
ax2.set_title('colors', fontsize=15)
fig.colorbar(im1, ax=ax1, shrink=0.7, aspect=30, orientation="horizontal", label="meters")
pyplot.show()

#### Visualize correlator confidence

In [ ]:
confidence_path = "outresults/dsmpre/confidence_from_ambiguity.tif"

confidence_pre_event = rasterio.open(confidence_path).read(1)
pyplot.figure(figsize = (15, 9))
pyplot.imshow(np.where(confidence_pre_event==255, np.nan, confidence_pre_event), cmap='viridis') # nodata value for confidence = 255
pyplot.title("Confidence pre-event")
pyplot.show()

#### Scatter (3D)

In [ ]:
import plotly.graph_objects as go

inds = np.random.choice(range(cloud.shape[0]), 20000)
x = cloud[inds, 0]
y = cloud[inds, 1]
z = cloud[inds, 2]

layout = go.Layout(scene=dict(aspectmode="data"),
                   height=600, width=1200,
                   title=go.layout.Title(text="Points Cloud Viewer"))
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers',
                                   marker=dict(
                                   size=2.5,
                                   color=z,
                                   colorscale='jet'))],
                layout=layout)
fig.show()

### Create your second configuration file

In [ ]:
import json
from cars.pipelines.pipeline import Pipeline

sensors = dict()
sensors["post1"] = {"image": "data/post_event_img1.tif",
                   "geomodel": "data/post_event_img1.geom",
                   "color": "data/post_event_color1.tif"}
sensors["post2"] = {"image": "data/post_event_img2.tif",
                   "geomodel": "data/post_event_img2.geom",
                   "color": "data/post_event_color2.tif"}

pairing = list()
pairing.append(["post1", "post2"])

inputs = {"sensors": sensors, "pairing": pairing}

config = {}
config["inputs"] = inputs
config["output"] = {"out_dir": "outresults/dsmpost"}
config["inputs"]["roi"] = {"type": "FeatureCollection",
                           "features": [{"type": "Feature", "properties": {},
                                         "geometry": {"coordinates":
                                                      [[[36.905, 37.588],
                                                        [36.905, 37.581],
                                                        [36.913, 37.581],
                                                        [36.913, 37.588],
                                                        [36.905, 37.588]]],
                                                      "type": "Polygon"}}]}
config["applications"] = {}
config["applications"]["point_cloud_rasterization"] = {"save_confidence": True}
config["applications"]["resampling"] = {"save_epipolar_image": True}

with open("config.json", "w") as f:
    json.dump(config, f, indent=2)

In [ ]:
!cat config.json

### Launch CARS - Command line

In [ ]:
!cars config.json

## Extract the Digital Terrain Models from Digital Surface Models through the Bulldozer library
and derive a final Digital Height Model based on the obtained results : https://github.com/cnes/bulldozer


### Launch Bulldozer - Python API

In [ ]:
from bulldozer.pipeline.bulldozer_pipeline import dsm_to_dtm

dsm_to_dtm(dsm_path="outresults/dsmpre/dsm.tif",
           output_dir="outresults/dtmpre",
           nb_max_workers=1,
           uniform_filter_size=3)

### Launch Bulldozer - Command line

In [ ]:
import yaml
config = {"dsm_path": "outresults/dsmpost/dsm.tif",
           "output_dir": "outresults/dtmpost",
           "nb_max_workers": 1,
           "uniform_filter_size": 3}

with open("config.yml", "w") as f:
    yaml.dump(config, f)

In [ ]:
!cat config.yml

In [ ]:
!bulldozer --conf config.yml

### Let's see the results

#### DSM, DTM, DHM profiles

In [ ]:
row_number = 600
start_col = 300
end_col = 1300
pair = "pre"

from rasterio.windows import Window
fig, axs = pyplot.subplots(3, 3, figsize=(20, 20))

gs = axs[1, 0].get_gridspec()

for ax in axs[1, 0:]:
    ax.remove()

for ax in axs[2, 0:]:
    ax.remove()

axdstm = fig.add_subplot(gs[1, 0:])
axdhm = fig.add_subplot(gs[2, 0:])

for idx, out in enumerate(["dsm"+pair+"/dsm.tif", "dtm"+pair+"/DTM.tif", "dtm"+pair+"/DHM.tif"]):
    with rasterio.open(os.path.join("outresults", out)) as src:
        axs[0, idx].plot([start_col, end_col], [row_number, row_number],
                         color="red", linewidth=4)
        profiley = np.ravel(src.read(1, window=Window(start_col, row_number, end_col-start_col, 1)))
        profilex = np.arange(len(profiley))
        profilex = profilex[profiley!=src.nodata]
        profiley = profiley[profiley!=src.nodata]

        if os.path.basename(out) != "DHM.tif":
            axdstm.plot(profilex, profiley, label=os.path.basename(out).lower())
        else:
            axdhm.plot(profilex, profiley, label=os.path.basename(out).lower())

        array = src.read(1)
        array[array==src.nodata] = np.nan
        axs[0, idx].imshow(array, cmap='gray')
        axs[0, idx].set_title(os.path.basename(out).lower())

axdstm.legend()
axdhm.legend()
pyplot.show()

## Extract semantic information from the 2D imagery: Building detection
by using classification models and spectral indices

### Download weights for building detection

In [ ]:
if os.path.isfile("weights.ckpt") is False:
    !gdown https://drive.google.com/uc?id=1n1olMUY3ycx48YRZ7ZG-ME63cNjnRBtc

### Launch inference

In [ ]:
if os.path.isdir("building_detection") is False:
    !mkdir building_detection
    !wget https://raw.githubusercontent.com/cars-cnes/change-detection-for-natural-disaster-response/main/building_detection/__init__.py -P building_detection

import building_detection
for pair in ["pre", "post"]:
    for idx in range(2):
        img = "data/"+pair+"_event_color"+str(idx+1)+".tif"
        with rasterio.open(img) as src:
            __, mask = building_detection.run(img, "weights.ckpt")
            profile = src.profile
            profile.update(
                dtype=rasterio.uint8,
                count=1,
                compress='lzw')
            with rasterio.open("outresults/buildings"+pair+str(idx+1)+".tif", 'w', nbits=1, **profile) as dst:
                dst.write((mask>0.5).astype(rasterio.uint8), 1)

### Let's see the results

In [ ]:
fig, axs = pyplot.subplots(2, 4, figsize=(20,10))
for idx1, pair in enumerate(["pre", "post"]):
    for idx2 in range(2):
        with rasterio.open("data/"+pair+"_event_color"+str(idx2+1)+".tif") as img, \
        rasterio.open("outresults/buildings"+pair+str(idx1+1)+".tif") as bld:
            show(normalize(img.read([1,2,3])/2**12), ax=axs[idx1, 2*idx2], title=pair+"-event "+str(idx2+1))
            show(bld, ax=axs[idx1, 2*idx2+1], title="buildings \n"+pair+"-event "+str(idx2+1))

### Project the masks in terrain geometry

In [ ]:
from rasterio import features
from shapely.geometry import Polygon
import geopandas as gpd
import pandas as pd

from shareloc.geofunctions import localization
from shareloc.image import Image
from shareloc.geomodels.geomodel import GeoModel

def polygonize(img):
    with rasterio.open(img) as src:
        mask = src.read(1)
        shapes = features.shapes(mask, mask=mask)
    fc = ({"geometry": shape, "properties": {"value": value}} for shape, value in shapes)
    return gpd.GeoDataFrame.from_features(fc)

def dsm_to_height_map(dsm, img):
    model = os.path.splitext(img)[0]+".geom"
    simage = Image(img, vertical_direction="north")
    smodel = GeoModel(model, "RPC")

    with rasterio.open(dsm) as src:
        crs = src.crs
        loc = localization.Localization(smodel, image=simage, epsg=crs.to_epsg())
        heights = src.read(1)
        cols, rows = np.meshgrid(np.arange(heights.shape[1]), np.arange(heights.shape[0]))
        xs, ys = rasterio.transform.xy(src.transform, rows, cols)
        pc = np.stack((np.array(xs), np.array(ys), np.array(heights)))
        pc = pc.reshape(3, -1).T
        pc = pc[pc[:, 2] != src.nodata]

    xyz = loc.inverse(pc[:, 0], pc[:, 1], pc[:, 2], using_geotransform=True)
    height_map = gpd.GeoDataFrame(pd.DataFrame({"height": xyz[2]}),
                                  geometry=gpd.points_from_xy(x=xyz[1], y=xyz[0]))
    return loc, crs, height_map

def set_altitude_to_polygons(polygons, height_map, loc, crs):
    height_map_join = gpd.sjoin(height_map, polygons, how="left")
    polygons_stats = height_map_join.groupby('index_right')['height'].agg(['mean','std','max','min'])
    polygons_with_alt = pd.merge(polygons, polygons_stats, left_index=True, right_index=True,how='outer')
    polygons_with_alt = polygons_with_alt.dropna()

    def project_polygon(poly, height):
        xx, yy = poly.exterior.coords.xy
        xyz = loc.direct(np.array(yy), np.array(xx), height, using_geotransform=True)
        xx = xyz[:, 0]
        yy = xyz[:, 1]
        return Polygon(zip(xx,yy))

    polygons_with_alt["geometry"] = polygons_with_alt.apply(lambda row : project_polygon(row["geometry"], row["mean"]), axis=1)
    polygons_with_alt.set_crs(crs)
    return polygons_with_alt

def burn_polygons(dsm, polygons, out):
  with rasterio.open(dsm) as src:
    profile = src.profile
    profile["dtype"] = rasterio.uint8
    profile["nbits"] = 1
    del profile["nodata"]
    with rasterio.open(out, 'w', **profile) as dst:
      image = features.rasterize(((g, 255) for g in polygons["geometry"]), transform=profile["transform"], out_shape=src.shape)
      dst.write(image, indexes=1)

fig, axs = pyplot.subplots(2, 2, figsize=(15,15))
for idx1, pair in enumerate(["pre", "post"]):
    dsm = "outresults/dsm"+pair+"/dsm.tif"
    for idx2 in range(2):
        msk = "outresults/buildings"+pair+str(idx2+1)+".tif"
        img = "data/"+pair+"_event_img"+str(idx2+1)+".tif"
        vectmsk = "outresults/geobuildings"+pair+str(idx2+1)+".shp"
        rastmsk = "outresults/geobuildings"+pair+str(idx2+1)+".tif"
        buildings = polygonize(msk)
        loc, crs, height_map = dsm_to_height_map(dsm, img)
        buildings_with_alt = set_altitude_to_polygons(buildings, height_map, loc, crs)
        buildings_with_alt.to_file(vectmsk)
        burn_polygons(dsm, buildings_with_alt, rastmsk)
        ax = axs[idx1, idx2]
        with rasterio.open(dsm) as raster:
            show(raster, ax=ax, title=pair+"-event "+str(idx2+1))
            buildings_with_alt.plot(ax=ax, facecolor='none', edgecolor='red')


## Change detection

### 2D Change detection

#### Raw 2D change detection

In [ ]:
buildings_pre_event_path_1 = "outresults/geobuildingspre1.tif"
buildings_pre_event_path_2 = "outresults/geobuildingspre2.tif"

buildings_pre_event_1 = rasterio.open(buildings_pre_event_path_1).read(1)
buildings_pre_event_2 = rasterio.open(buildings_pre_event_path_2).read(1)
buildings_pre_event_merged = np.logical_or(buildings_pre_event_1, buildings_pre_event_2)

buildings_post_event_path_1 = "outresults/geobuildingspost1.tif"
buildings_post_event_path_2 = "outresults/geobuildingspost2.tif"

buildings_post_event_1 = rasterio.open(buildings_post_event_path_1).read(1)
buildings_post_event_2 = rasterio.open(buildings_post_event_path_2).read(1)
buildings_post_event_merged = np.logical_or(buildings_post_event_1, buildings_post_event_2)

fig1, (ax1, ax2) = pyplot.subplots(1,2, sharex=True, sharey=True,  figsize=(15,9))
ax1.imshow(buildings_pre_event_merged)
ax2.imshow(buildings_post_event_merged)

ax1.title.set_text('Building detection pre-event')
ax2.title.set_text('Building detection  post-event')
pyplot.show()

In [ ]:
pyplot.figure(figsize = (15, 9))
raw_diff_2D = np.logical_and(buildings_pre_event_merged, np.logical_not(buildings_post_event_merged))
pyplot.imshow(raw_diff_2D)
pyplot.title("Raw 2D change detection")
pyplot.show()

#### Refinement

In [ ]:
refined_2D_diff = opening_by_reconstruction(raw_diff_2D.astype(np.uint8), kernel).astype(bool)
# Small object filter (>~12m²)
refined_2D_diff = morphology.remove_small_objects(refined_2D_diff.astype(bool), 500)

pyplot.figure(figsize = (15, 9))
pyplot.imshow(refined_2D_diff)
pyplot.title("Refined 2D change detection (morphologic filter)")
pyplot.show()

#### 2D change detection visualization

In [ ]:
fig1, (ax1, ax2, ax3) = pyplot.subplots(1,3, sharex=True, sharey=True,  figsize=(25,9))
# pre-event
ax1.imshow(normalize(pre_event_colors))
# post-event
color_path_post_event = "outresults/dsmpost/clr.tif"
post_event_colors = rasterio.open(color_path_post_event).read()[:3,:,:].transpose(1,2,0)
post_event_colors = normalize(post_event_colors)
ax2.imshow(post_event_colors)
# 2D change detection
post_event_colors[refined_2D_diff, 0] = 255
post_event_colors[refined_2D_diff, 1:] = 0
ax3.imshow(post_event_colors)

ax1.title.set_text('True ortho pre-event')
ax2.title.set_text('True ortho post-event')
ax3.title.set_text('True ortho post-event + 2D change labels')
pyplot.show()

### 3D Change detection

#### Raw difference

In [ ]:
dsm_pre_event_path = "outresults/dsmpre/dsm.tif"
dsm_post_event_path = "outresults/dsmpost/dsm.tif"
dsm_pre_event = rasterio.open(dsm_pre_event_path).read(1)
dsm_post_event = rasterio.open(dsm_post_event_path).read(1)

fig1, (ax1, ax2) = pyplot.subplots(1,2, sharex=True, sharey=True,  figsize=(15,9))
ax1.imshow(np.where(dsm_pre_event==nodata, np.nan, dsm_pre_event), cmap='viridis')
ax2.imshow(np.where(dsm_post_event==nodata, np.nan, dsm_post_event), cmap='viridis')

ax1.title.set_text('DSM pre-event')
ax2.title.set_text('DSM post-event')
pyplot.show()


In [ ]:
raw_diff_3D = dsm_post_event - dsm_pre_event
raw_diff_3D[np.logical_or(dsm_pre_event == nodata, dsm_post_event == nodata)] = nodata
pyplot.figure(figsize = (15, 9))
pyplot.imshow(np.where(raw_diff_3D==nodata, np.nan, raw_diff_3D), cmap='viridis')
pyplot.title("Raw 3D difference")
pyplot.show()


In [ ]:
pyplot.figure(figsize = (15, 9))
pyplot.imshow(np.where(np.logical_or(raw_diff_3D==nodata, raw_diff_3D > 0), 0, 1), cmap='viridis')
pyplot.title("Raw binary 3D destruction (difference < 0m)")
pyplot.show()

#### Detection refinement

Altimetric filter

In [ ]:
refined_3D_diff = raw_diff_3D.copy()
# Altimetric filter
altimetric_resolution = 2 * 0.5 # altimetric_resolution in meter. We approximate the altimetric resolution as = 2*plannimetric_resolution
z_treshold = 2 * altimetric_resolution
refined_3D_diff[raw_diff_3D > -z_treshold] = nodata

#Binarization
bin_refined_3D_diff = np.full((raw_diff_3D.shape[0], raw_diff_3D.shape[1]), False)
bin_refined_3D_diff[refined_3D_diff != nodata] = 1
bin_refined_3D_diff[refined_3D_diff == nodata] = 0

pyplot.figure(figsize = (15, 9))
pyplot.imshow(bin_refined_3D_diff)
pyplot.title("Filtered binary difference (altimetric filter)")
pyplot.show()

Vegetation filter

In [ ]:
# Vegetation filter (NDVI)
color_path_pre_event = "outresults/dsmpre/clr.tif"
color_path_post_event = "outresults/dsmpost/clr.tif"

ndvi_threshold = 0.4

# ignore divide warning in NDVI computation
np.seterr(divide='ignore', invalid='ignore')

# Pre-event filtering
with rasterio.open(color_path_pre_event) as color_pre_event_dataset:
    color = color_pre_event_dataset.read()
    ndvi = (color[3,:,:] - color[2, :,:])/(color[3,:,:] + color[2,:,:])
    bin_refined_3D_diff[ndvi > ndvi_threshold] = 0

# Post-event filtering
with rasterio.open(color_path_post_event) as color_post_event_dataset:
    color = color_post_event_dataset.read()
    ndvi = (color[3,:,:] - color[2, :,:])/(color[3,:,:] + color[2,:,:])
    bin_refined_3D_diff[ndvi > ndvi_threshold] = 0

pyplot.figure(figsize = (15, 9))
pyplot.imshow(bin_refined_3D_diff)
pyplot.title("Filtered binary difference (altimetric+vegetation filters)")
pyplot.show()

Confidence filter


In [ ]:
# confidence filter
confidence_path_pre_event = "outresults/dsmpre/confidence_from_ambiguity.tif"
confidence_path_post_event = "outresults/dsmpost/confidence_from_ambiguity.tif"

confidence_threshold = 0.6

# Pre-event filtering
with rasterio.open(confidence_path_pre_event) as confidence_pre_event_dataset:
    confidence = confidence_pre_event_dataset.read(1)
    bin_refined_3D_diff[confidence < confidence_threshold] = 0

# Post-event filtering
with rasterio.open(confidence_path_post_event) as confidence_post_event_dataset:
    confidence = confidence_post_event_dataset.read(1)
    bin_refined_3D_diff[confidence < confidence_threshold] = 0

pyplot.figure(figsize = (15, 9))
pyplot.imshow(bin_refined_3D_diff)
pyplot.title("Filtered binary difference (altimetric+vegetation+confidence filters)")
pyplot.show()

Morphologic filter


In [ ]:
bin_refined_3D_diff = opening_by_reconstruction(bin_refined_3D_diff.astype(np.uint8), kernel)
# Small object filter (>~12m²)
bin_refined_3D_diff = morphology.remove_small_objects(bin_refined_3D_diff.astype(bool), 500)

pyplot.figure(figsize = (15, 9))
pyplot.imshow(bin_refined_3D_diff)
pyplot.title("Filtered binary difference (altimetric+vegetation+confidence+morphologic filters)")
pyplot.show()

3D Change detection visualization

In [ ]:
fig1, (ax1, ax2, ax3) = pyplot.subplots(1,3, sharex=True, sharey=True,  figsize=(25,9))
# pre-event
ax1.imshow(normalize(pre_event_colors))
# post-event
color_path_post_event = "outresults/dsmpost/clr.tif"
post_event_colors = normalize(rasterio.open(color_path_post_event).read()[:3,:,:].transpose(1,2,0))
ax2.imshow(post_event_colors)
# 3D change detection
post_event_colors[bin_refined_3D_diff, 0] = 255
post_event_colors[bin_refined_3D_diff, 1:] = 0
ax3.imshow(post_event_colors)

ax1.title.set_text('True ortho pre-event')
ax2.title.set_text('True ortho post-event')
ax3.title.set_text('True ortho post-event + 3D change labels')
pyplot.show()

### 2D and 3D change comparison

In [ ]:
fig1, (ax1, ax2) = pyplot.subplots(1,2, sharex=True, sharey=True,  figsize=(15,9))
ax1.imshow(refined_2D_diff)
ax2.imshow(bin_refined_3D_diff)

ax1.title.set_text('Filtered 2D Changes')
ax2.title.set_text('Filtered 3D Changes')
pyplot.show()